In [6]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import Loss
def make_aipw(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')
    # representation
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #propensity prediction
    #Note that the activation is actually sigmoid, but we will squish it in the loss function for numerical stability reasons
    t_prediction = Dense(units=1,activation=None,name='t_prediction')(phi)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_prediction,phi])
    model = Model(inputs=x, outputs=concat_pred)
    return model

class Base_Loss(Loss):
    #initialize instance attributes
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.name='standard_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    #for logging purposes only
    def treatment_acc(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        #Since this isn't used as a loss, I've used tf.reduce_mean for interpretability
        return tf.reduce_mean(binary_accuracy(t_true, tf.math.sigmoid(p['t_pred']), threshold=0.5))

    def treatment_bce(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        lossP = tf.reduce_sum(binary_crossentropy(t_true,p['t_pred'],from_logits=True))
        return lossP
    
    def regression_loss(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - p['y0_pred']))
        loss1 = tf.reduce_sum(t_true * tf.square(y_true - p['y1_pred']))
        return loss0+loss1

    def standard_loss(self,concat_true,concat_pred):
        lossR = self.regression_loss(concat_true,concat_pred)
        lossP = self.treatment_bce(concat_true,concat_pred)
        return lossR + self.alpha * lossP

    #compute loss
    def call(self, concat_true, concat_pred):        
        return self.standard_loss(concat_true,concat_pred)
        

In [7]:
from tensorflow.keras.callbacks import Callback

def pdist2sq(A, B):
    #helper for PEHEnn
    #calculates squared euclidean distance between rows of two matrices  
    #https://gist.github.com/mbsariyildiz/34cdc26afb630e8cae079048eef91865
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)    
    # na as a row and nb as a column vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    # return pairwise euclidean difference matrix
    D=tf.reduce_sum((tf.expand_dims(A, 1)-tf.expand_dims(B, 0))**2,2) 
    return D

from evaluation import Full_Metrics, metrics_for_cevae

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class AIPW_Metrics(Callback):
    def __init__(self,data,name='train', verbose=0):   
        super(AIPW_Metrics, self).__init__()
        self.data=data #feed the callback the full dataset
        self.verbose=verbose
        self.name = name

        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units
    
    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(np.reshape(concat_pred[:, 0],[-1,1]))[:,0]
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(np.reshape(concat_pred[:, 1],[-1,1]))[:,0]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    def find_ynn(self, Phi):
        #helper for PEHEnn
        PhiC, PhiT =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(self.data['t']),tf.int32),2) #separate control and treated reps
        dists=tf.sqrt(pdist2sq(PhiC,PhiT)) #calculate squared distance then sqrt to get euclidean
        yT_nn_idx=tf.gather(self.data['c_idx'],tf.argmin(dists,axis=0),1) #get c_idxs of smallest distances for treated units
        yC_nn_idx=tf.gather(self.data['t_idx'],tf.argmin(dists,axis=1),1) #get t_idxs of smallest distances for control units
        yT_nn=tf.gather(self.data['y'],yT_nn_idx,1) #now use these to retrieve y values
        yC_nn=tf.gather(self.data['y'],yC_nn_idx,1)
        y_nn=tf.dynamic_stitch([self.data['t_idx'],self.data['c_idx']],[yT_nn,yC_nn]) #stitch em back up!
        return y_nn

    def PEHEnn(self,concat_pred):
        p = self.split_pred(concat_pred)
        y_nn = self.find_ynn(p['phi']) #now its 3 plus because 
        cate_nn_err=tf.reduce_mean( tf.square( (1-2*self.data['t']) * (y_nn-self.data['y']) - (p['y1_pred']-p['y0_pred']) ) )
        return cate_nn_err

    def ATE(self,concat_pred):
        p = self.split_pred(concat_pred)
        return p['y1_pred']-p['y0_pred']

    def PEHE(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (p['y1_pred']-p['y0_pred']) ) ) )
        return cate_err 
   
    #THIS IS THE NEW PART
    def AIPW(self,concat_pred):
        p = self.split_pred(concat_pred)
        t_pred=tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        y_pred = p['y0_pred'] * (1 - self.data['t']) + p['y1_pred'] * self.data['t']
        #cc stands for clever covariate which is I think what it's called in TMLE lit
        cc = self.data['t'] * (1.0 / p['t_pred']) - (1.0 - self.data['t']) / (1.0 - p['t_pred'])
        cate = cc * (self.data['y'] - y_pred) + p['y1_pred'] - p['y0_pred']
        return cate
    
    def rmse(self,concat_pred):
        p = self.split_pred(concat_pred)
        y_pred = p['y0_pred'] * (1 - self.data['t']) + p['y1_pred'] * self.data['t']
        return tf.math.square(self.data['y']-y_pred)
    def regret(self,concat_pred):
        p = self.split_pred(concat_pred)
        t_pred=tf.math.sigmoid(p['t_pred'])
        y_pred = p['y0_pred'] * (1 - t_pred) + p['y1_pred'] * t_pred
        return tf.math.square(self.data['y']-y_pred)

    def on_epoch_end(self, epoch, logs={}):
        concat_pred=self.model.predict(self.data['x'])
        #Calculate Empirical Metrics        
        ate_pred=tf.reduce_mean(self.ATE(concat_pred)); tf.summary.scalar(f'{self.name}_ate', data=ate_pred, step=epoch)
        pehe_nn=self.PEHEnn(concat_pred); tf.summary.scalar(f'{self.name}_cate_nn_err', data=tf.sqrt(pehe_nn), step=epoch)
        aipw=tf.reduce_mean(self.AIPW(concat_pred)); tf.summary.scalar(f'{self.name}_aipw', data=aipw, step=epoch)
        rmse = tf.reduce_mean(self.rmse(concat_pred)); tf.summary.scalar(f'{self.name}_rmse', data=aipw, step=epoch)
        regret = tf.reduce_mean(self.regret(concat_pred)); tf.summary.scalar(f'{self.name}_regret', data=aipw, step=epoch)
        #Simulation Metrics
        ate_true=tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        ate_err=tf.abs(ate_true-ate_pred); tf.summary.scalar(f'{self.name}_ate_err', data=ate_err, step=epoch)
        pehe =self.PEHE(concat_pred); tf.summary.scalar(f'{self.name}_cate_err', data=tf.sqrt(pehe), step=epoch)
        aipw_err =self.PEHE(concat_pred); tf.summary.scalar(f'{self.name}_aipw_err', data=aipw_err, step=epoch)
        out_str=f' — ate_err: {ate_err:.4f}  — aipw_err: {aipw_err:.4f} — cate_err: {tf.sqrt(pehe):.4f} — cate_nn_err: {tf.sqrt(pehe_nn):.4f} - rmse:{rmse:.4f} - regret:{regret:.4f} '
        
        if self.verbose > 0: print(out_str)

In [8]:
#@title First load the data! (Click Play)
import numpy as np
from sklearn.preprocessing import StandardScaler
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.train.npz
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.test.npz 

def load_IHDP_data(training_data,testing_data,i=7):
    with open(training_data,'rb') as trf, open(testing_data,'rb') as tef:
        train_data=np.load(trf); test_data=np.load(tef)
        y=np.concatenate(   (train_data['yf'][:,i],   test_data['yf'][:,i])).astype('float32') #most GPUs only compute 32-bit floats
        t=np.concatenate(   (train_data['t'][:,i],    test_data['t'][:,i])).astype('float32')
        x=np.concatenate(   (train_data['x'][:,:,i],  test_data['x'][:,:,i]),axis=0).astype('float32')
        mu_0=np.concatenate((train_data['mu0'][:,i],  test_data['mu0'][:,i])).astype('float32')
        mu_1=np.concatenate((train_data['mu1'][:,i],  test_data['mu1'][:,i])).astype('float32')

        data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
        data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
        data['y']=data['y'].reshape(-1,1)
        
        #rescaling y between 0 and 1 often makes training of DL regressors easier
        data['y_scaler'] = StandardScaler().fit(data['y'])
        data['ys'] = data['y_scaler'].transform(data['y'])

    return data

data_train=load_IHDP_data(training_data='./ihdp_npci_1-100.train.npz',testing_data='./ihdp_npci_1-100.train.npz')
data_valid=load_IHDP_data(training_data='./ihdp_npci_1-100.test.npz',testing_data='./ihdp_npci_1-100.test.npz')
np.shape(data_train['x'])

文件 “ihdp_npci_1-100.train.npz” 已经存在；不获取。

文件 “ihdp_npci_1-100.test.npz” 已经存在；不获取。



(1344, 25)

In [9]:
import tensorflow as tf
import numpy as np
import datetime
%load_ext tensorboard

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam

val_split=0.2
batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)

# Clear any logs from previous runs
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
        tensorboard_callback,
        Full_Metrics(data_train,'train',verbose=verbose),
        # AIPW_Metrics(data_valid,'valid',verbose=verbose),
        Full_Metrics(data_valid,'valid',verbose=verbose),
        ]
      

sgd_lr = 1e-5
momentum = 0.9

aipw_model=make_aipw(data_train['x'].shape[1],.01)
aipw_loss=Base_Loss(alpha=1.0)

aipw_model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                    loss=aipw_loss,
                    metrics=[aipw_loss,aipw_loss.regression_loss,aipw_loss.treatment_acc]
                   )

aipw_model.fit(
        x=data_train['x'],
        y=np.concatenate([data_train['ys'], data_train['t']], 1),
        callbacks=sgd_callbacks,
        validation_data=[data_valid['x'],np.concatenate([data_valid['ys'], data_valid['t']], 1)],
        epochs=300,
        batch_size=batch_size,
        verbose=verbose)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/300
21/21 [==============================] - 3s 79ms/step - loss: 62.5602 - standard_loss: 57.8595 - regression_loss: 57.1628 - treatment_acc: 0.4821 - val_loss: 45.7890 - val_standard_loss: 36.1322 - val_regression_loss: 35.4342 - val_treatment_acc: 0.3807 - lr: 1.0000e-05
Epoch 2/300
21/21 [==============================] - 1s 33ms/step - loss: 42.2002 - standard_loss: 37.4991 - regression_loss: 36.8031 - treatment_acc: 0.4814 - val_loss: 32.3338 - val_standard_loss: 24.4891 - val_regression_loss: 23.7900 - val_treatment_acc: 0.4010 - lr: 1.0000e-05
Epoch 3/300
21/21 [==============================] - 1s 35ms/step - loss: 32.2097 - standard_loss: 27.5076 - regression_loss: 26.8122 - treatment_acc: 0.4903 - val_loss: 26.8289 - val_standard_loss: 19.6331 - val_regression_loss: 18.9353 - val_treatment_acc: 0.4626 - lr: 1.0000e-05
Epoch 4/300
21/21 [==============================] - 1s 32ms/step - loss: 27.9789 - standard_loss: 23.2757 - regression_loss: 22.5822 - treatment_acc: 

In [10]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 45269), started 2 days, 0:53:37 ago. (Use '!kill 45269' to kill it.)